# ATTENTION

In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, GlobalMaxPooling1D, Dropout, Conv1D, GlobalAveragePooling1D, LayerNormalization #Remove
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, GlobalMaxPooling1D, Dropout, Conv1D, GlobalAveragePooling1D, LayerNormalization, MultiHeadAttention, Input, Flatten, Add, TimeDistributed, Concatenate
from keras.models import Model
from tensorflow.keras.regularizers import l2
from keras.optimizers import AdamW

In [3]:
from intent_recognition import *

In [8]:
hyperparams = {'vocab_size': 500, 'embedding_dim': 1024, 'epochs': 15, 'batch_size': 32}
train_config = {'use_class_weights': True}
temp_model = Sequential()
temp_model = IntentRecognition(model=temp_model, hyperparams=hyperparams, train_config=train_config)
train_pad_sequences = temp_model.train_pad_sequences
test_pad_sequences = temp_model.test_pad_sequences
val_pad_sequences = temp_model.val_pad_sequences
train_encoded_labels = temp_model.train_encoded_labels
test_encoded_labels = temp_model.test_encoded_labels
val_encoded_labels = temp_model.val_encoded_labels

In [11]:
#TODO
vocab_size = 500 + 1
key_dim = 32
embedding_dim = 1024
block_size = 45
l2_lambda = 0
attention_blocks = 2

inputs = Input(shape=(block_size,))
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(inputs)
positions = Embedding(input_dim=block_size, output_dim=embedding_dim)(tf.range(start=0, limit=block_size))
positions = tf.expand_dims(positions, axis=0)
x = Add()([embedding, positions])


x = Conv1D(128, 2)(x)

x = GlobalMaxPooling1D()(x)

output = Dense(64)(x)
output = Dense(22, activation='softmax')(x)  # Output layer for classification (22 classes)

# Define the model
model = Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', F1Score(average='macro')])

# Train the model
batch_size = 16
epochs = 20
model.fit(train_pad_sequences, train_encoded_labels, batch_size=batch_size, epochs=epochs, validation_data=(val_pad_sequences, val_encoded_labels))
# Evaluate the model on the test set
loss, accuracy, f1 = model.evaluate(test_pad_sequences, test_encoded_labels, batch_size=batch_size)
print(f"Test accuracy: {accuracy:.4f}")

Epoch 1/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7910 - f1_score: 0.1494 - loss: 0.8987 - val_accuracy: 0.9333 - val_f1_score: 0.3352 - val_loss: 0.2474
Epoch 2/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9718 - f1_score: 0.4808 - loss: 0.1143 - val_accuracy: 0.9656 - val_f1_score: 0.5836 - val_loss: 0.1664
Epoch 3/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9911 - f1_score: 0.6277 - loss: 0.0471 - val_accuracy: 0.9667 - val_f1_score: 0.6261 - val_loss: 0.1562
Epoch 4/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9964 - f1_score: 0.7247 - loss: 0.0182 - val_accuracy: 0.9678 - val_f1_score: 0.6155 - val_loss: 0.1316
Epoch 5/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9989 - f1_score: 0.8497 - loss: 0.0098 - val_accuracy: 0.9711 - val_f1_score: 0.6228 - val_loss: 0.1278
Epoch 6/20
255/255 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9990 - f1_score: 0.8375 - loss: 0.0044 - val_accuracy: 0.9689 - val_f1_score: 0.60